東京２３区の各区における平均地価の外部データを取得したので、それを使ってモデルを動かしてみる！

In [284]:
import pandas as pd

２３区のデータ取り込み

In [285]:
df_adachi = pd.read_csv('../data/handmade_data/block_aveprice_adachi2019.csv')
df_arakawa = pd.read_csv('../data/handmade_data/block_aveprice_arakawa2019.csv')
df_bunkyo = pd.read_csv('../data/handmade_data/block_aveprice_bunkyo2019.csv')
df_chiyoda = pd.read_csv('../data/handmade_data/block_aveprice_chiyoda2019.csv')
df_chuo = pd.read_csv('../data/handmade_data/block_aveprice_chuo2019.csv')
df_edogawa = pd.read_csv('../data/handmade_data/block_aveprice_edogawa2019.csv')
df_itabashi = pd.read_csv('../data/handmade_data/block_aveprice_itabashi2019.csv')
df_katsushika = pd.read_csv('../data/handmade_data/block_aveprice_katsushika2019.csv')
df_kita = pd.read_csv('../data/handmade_data/block_aveprice_kita2019.csv')
df_koto = pd.read_csv('../data/handmade_data/block_aveprice_koto2019.csv')
df_meguro = pd.read_csv('../data/handmade_data/block_aveprice_meguro2019.csv')
df_minato = pd.read_csv('../data/handmade_data/block_aveprice_minato2019.csv')
df_nakano = pd.read_csv('../data/handmade_data/block_aveprice_nakano2019.csv')
df_nerima = pd.read_csv('../data/handmade_data/block_aveprice_nerima2019.csv')
df_ota = pd.read_csv('../data/handmade_data/block_aveprice_ota2019.csv')
df_setagaya = pd.read_csv('../data/handmade_data/block_aveprice_setagaya2019.csv')
df_shibuya = pd.read_csv('../data/handmade_data/block_aveprice_shibuya2019.csv')
df_shinagawa = pd.read_csv('../data/handmade_data/block_aveprice_shinagawa2019.csv')
df_shinjuku = pd.read_csv('../data/handmade_data/block_aveprice_shinjuku2019.csv')
df_suginami = pd.read_csv('../data/handmade_data/block_aveprice_suginami2019.csv')
df_sumida = pd.read_csv('../data/handmade_data/block_aveprice_sumida2019.csv')
df_taito = pd.read_csv('../data/handmade_data/block_aveprice_taito2019.csv')
df_toshima = pd.read_csv('../data/handmade_data/block_aveprice_toshima2019.csv')

df_23wards = pd.concat([
    df_adachi,
    df_arakawa,
    df_bunkyo,
    df_chiyoda,
    df_chuo,
    df_edogawa,
    df_itabashi,
    df_katsushika,
    df_kita,
    df_koto,
    df_meguro,
    df_minato,
    df_nakano,
    df_nerima,
    df_ota,
    df_setagaya,
    df_shibuya,
    df_shinjuku,
    df_shinagawa,
    df_suginami,
    df_sumida,
    df_taito,
    df_toshima
],ignore_index=True)
#df_23wards.columns = [['区町名', '平均地価']]
#df_23wards = df_23wards[['区町名', '平均地価']]

In [286]:
df_23wards

,0,1
0,足立区一ツ家,NaN
1,足立区中央本町,335000.0
2,足立区中川,320500.0
3,足立区伊興,318000.0
4,足立区伊興本町,256500.0
...,...,...
740,豊島区長崎,587500.0
741,豊島区雑司が谷,570000.0
742,豊島区駒込,807600.0
743,豊島区高松,520000.0


SIGNATE側から提供されていた土地データとマージする

In [287]:
addresses = pd.read_csv('../data/handmade_data/merge_address.csv')
addresses = addresses['所在地']
addresses = pd.DataFrame(addresses)
#addresses.columns = ['区町名']

In [288]:
address_aveprice = pd.merge(addresses, df_23wards, left_on='所在地', right_on='0', how='left')
address_aveprice = address_aveprice[['所在地', '1']]
address_aveprice.columns = [['区町名', '平均地価']]

このデータを他のLGBDに入力するデータと合体させる

In [289]:
df = pd.read_csv('../data/outer_data/train.csv')
dummy_address_aveprice = address_aveprice['平均地価']
land_aveprice = dummy_address_aveprice.iloc[:len(df),:]
test_land_aveprice = dummy_address_aveprice.iloc[len(df):,:]
land_aveprice.to_csv('../data/handmade_data/land_aveprice.csv')
test_land_aveprice.to_csv('../data/test_data/test_land_aveprice.csv')

In [290]:
land_aveprice.columns = ['平均地価']
test_land_aveprice.columns = ['平均地価']

In [280]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import scipy as sp
import lightgbm as lgb
import category_encoders as ce
# import mojimoji
import re
from cmath import nan
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score
import optuna
from sklearn.metrics import mean_squared_error
import japanize_matplotlib
from sklearn.model_selection import KFold

###訓練データの読み込み
house_age = pd.read_csv('../data/handmade_data/house_age.csv')
area_size = pd.read_csv('../data/handmade_data/area_size.csv')
room_arrange = pd.read_csv('../data/handmade_data/room_arrange.csv')
contract_span = pd.read_csv('../data/handmade_data/contract_span.csv')#欠損値をゼロ埋めした契約期間
contract_span2 = pd.read_csv('../data/handmade_data/contract_span2.csv')#欠損値を欠損値のままにした契約期間
reg_rent = pd.read_csv('../data/handmade_data/reg_rent.csv')
floor_scores = pd.read_csv('../data/handmade_data/floor_scores.csv')#「所在階」と「全体の階数」が連動して欠損値になる場合のある「所在階」
floor_scores2 = pd.read_csv('../data/handmade_data/floor_scores2.csv')#「所在階」と「全体の階数」が独立している「所在階」
Floor_scores = pd.read_csv('../data/handmade_data/capital_floor_scores.csv')#「所在階」と「全体の階数」が連動して欠損値になる場合のある「全体の階数」
Floor_scores2 = pd.read_csv('../data/handmade_data/capital_floor_scores2.csv')#「所在階」と「全体の階数」が独立している「全体の階数」
stations = pd.read_csv('../data/handmade_data/stations.csv')
minits = pd.read_csv('../data/handmade_data/minits.csv')
addresses = pd.read_csv('../data/handmade_data/addresses.csv')
room_arrange = pd.read_csv('../data/handmade_data/room_arrange.csv')
buildings = pd.read_csv('../data/handmade_data/buildings.csv')

rent = pd.read_csv('../data/handmade_data/rent.csv')


##テストデータの読み込み
test_house_age = pd.read_csv('../data/test_data/test_house_age.csv')
test_area_size = pd.read_csv('../data/test_data/test_area_size.csv')
test_room_arrange = pd.read_csv('../data/test_data/test_room_arrange.csv')
test_contract_span = pd.read_csv('../data/test_data/test_contract_span.csv')
test_contract_span2 = pd.read_csv('../data/test_data/test_contract_span2.csv')
test_reg_rent = pd.read_csv('../data/test_data/test_reg_rent.csv')
test_floor_scores = pd.read_csv('../data/test_data/test_floor_scores.csv')
test_floor_scores2 = pd.read_csv('../data/test_data/test_floor_scores2.csv')
test_Floor_scores = pd.read_csv('../data/test_data/test_capital_floor_scores.csv')
test_Floor_scores2 = pd.read_csv('../data/test_data/test_capital_floor_scores2.csv')
test_stations = pd.read_csv('../data/test_data/test_stations.csv')
test_minits = pd.read_csv('../data/test_data/test_minits.csv')
test_addresses = pd.read_csv('../data/test_data/test_addresses.csv')
test_room_arrange = pd.read_csv('../data/test_data/test_room_arrange.csv')
test_buildings = pd.read_csv('../data/test_data/test_buildings.csv')

In [281]:
X_train = pd.concat([house_age, area_size, contract_span2, reg_rent, floor_scores2, Floor_scores2, stations, minits, land_aveprice, room_arrange[["部屋数","L","D","K","S"]]], axis=1)
y_train = rent
X_test = pd.concat([test_house_age, test_area_size, test_contract_span2, test_reg_rent, test_floor_scores2, test_Floor_scores2, test_stations, test_minits, test_land_aveprice, test_room_arrange[["部屋数","L","D","K","S"]]], axis=1)
category_lists = ['定期借家','最寄り駅', 'L', 'D','K','S']

In [282]:
X_train

,築年数,面積,契約期間,定期借家,所在階,全体の階数,最寄り駅,所要時間,平均地価,部屋数,L,D,K,S
0,117.0,20.01,2.0,1,1.0,12.0,1,4.0,6.016250e+05,1,0,0,1,0
1,538.0,16.50,2.0,1,5.0,10.0,2,5.0,1.408000e+06,1,0,0,0,0
2,102.0,22.05,2.0,1,12.0,15.0,3,6.0,8.550000e+05,1,0,0,1,0
3,352.0,60.48,2.0,2,3.0,4.0,4,9.0,6.988750e+05,2,1,1,1,0
4,379.0,39.66,2.0,1,1.0,2.0,5,5.0,3.416667e+05,2,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31465,360.0,37.90,2.0,1,1.0,3.0,223,7.0,4.200000e+05,2,0,1,1,0
31466,475.0,12.04,2.0,1,4.0,4.0,118,6.0,1.251333e+06,1,0,0,0,0
31467,550.0,60.00,2.0,1,3.0,5.0,91,5.0,4.428571e+05,3,0,1,1,0
31468,132.0,29.59,2.0,1,1.0,2.0,73,4.0,1.558583e+06,1,0,1,1,0


In [249]:
cv = KFold(n_splits=5, shuffle=True, random_state=0)
for fold_id, (train_index, valid_index) in enumerate(cv.split(X_train)):
    X_tr = X_train.loc[train_index,:]
    X_val = X_train.loc[valid_index, :]
    y_tr = y_train.loc[train_index]
    y_val = y_train.loc[valid_index]

    lgb_train = lgb.Dataset(X_tr, y_tr)
    lgb_eval = lgb.Dataset(X_val, y_val, reference=lgb_train)

    params = {
        'objective':'regression',
        'metrics':'rmse',
        'learning_rate':0.05,
        'reg_lambda': 4.430375245218262e-06,
        'max_bin': 502,
        'num_leaves': 97
    }

    model = lgb.train(
                        params,
                        lgb_train, 
                        valid_sets=[lgb_train, lgb_eval], 
                        verbose_eval=10, 
                        num_boost_round=1000, 
                        early_stopping_rounds=10,
                        categorical_feature = category_lists
                        )

y_pred = model.predict(X_test, num_iteration=model.best_iteration)
# feature importanceを表示
importance = pd.DataFrame(model.feature_importance(importance_type = "gain"), index=X_train.columns, columns=['importance'])
df_importance = pd.DataFrame(importance)
df_importance = df_importance.sort_values('importance', ascending=False) # 降順ソート
display(df_importance)


/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['D', 'K', 'L', 'S', '定期借家', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument 

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001724 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2087
[LightGBM] [Info] Number of data points in the train set: 25176, number of used features: 14
[LightGBM] [Info] Start training from score 118516.069987
Training until validation scores don't improve for 10 rounds
[10]	training's rmse: 48825.6	valid_1's rmse: 52082.5
[20]	training's rmse: 34524.9	valid_1's rmse: 40394.8
[30]	training's rmse: 26214.1	valid_1's rmse: 34203.7
[40]	training's rmse: 21069.2	valid_1's rmse: 30765.7
[50]	training's rmse: 17930.2	valid_1's rmse: 28811.3
[60]	training's rmse: 16072.1	valid_1's rmse: 27695.2
[70]	training's rmse: 14724.7	valid_1's rmse: 26889.9
[80]	training's rmse: 13821.2	valid_1's rmse: 26402.4
[90]	training's rmse: 13123.4	valid_1's rmse: 26018.3
[100]	training's rmse: 12502.2	valid_1'

/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['D', 'K', 'L', 'S', '定期借家', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument 

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000773 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2088
[LightGBM] [Info] Number of data points in the train set: 25176, number of used features: 14
[LightGBM] [Info] Start training from score 118478.809819
Training until validation scores don't improve for 10 rounds
[10]	training's rmse: 49473	valid_1's rmse: 49579.1
[20]	training's rmse: 35601.4	valid_1's rmse: 36073.9
[30]	training's rmse: 27772.3	valid_1's rmse: 28638.7
[40]	training's rmse: 23301.4	valid_1's rmse: 24340.9
[50]	training's rmse: 20542.8	valid_1's rmse: 21621
[60]	training's rmse: 18965.2	valid_1's rmse: 20218.9
[70]	training's rmse: 17858.6	valid_1's rmse: 19554.6
[80]	training's rmse: 17114.6	valid_1's rmse: 19173.3
[90]	training's rmse: 16419.5	valid_1's rmse: 18883.5
[100]	training's rmse: 15878.8	valid_1's rm

/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['D', 'K', 'L', 'S', '定期借家', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument 

...

/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['D', 'K', 'L', 'S', '定期借家', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument 

[30]	training's rmse: 27509.5	valid_1's rmse: 28110.9
[40]	training's rmse: 22897.3	valid_1's rmse: 24365.8
[50]	training's rmse: 20009.7	valid_1's rmse: 22440.2
[60]	training's rmse: 18374.5	valid_1's rmse: 21468.3
[70]	training's rmse: 17262.3	valid_1's rmse: 20727
[80]	training's rmse: 16575.7	valid_1's rmse: 20227
[90]	training's rmse: 15942.1	valid_1's rmse: 19834.1
[100]	training's rmse: 15464.6	valid_1's rmse: 19553.2
[110]	training's rmse: 14990.7	valid_1's rmse: 19305
[120]	training's rmse: 14614.1	valid_1's rmse: 19136.1
[130]	training's rmse: 14276.4	valid_1's rmse: 19012.3
[140]	training's rmse: 13944.4	valid_1's rmse: 18843.8
[150]	training's rmse: 13667.4	valid_1's rmse: 18785.1
[160]	training's rmse: 13391.6	valid_1's rmse: 18659.7
[170]	training's rmse: 13175.4	valid_1's rmse: 18619.2
[180]	training's rmse: 12939.4	valid_1's rmse: 18512.3
[190]	training's rmse: 12757.9	valid_1's rmse: 18431.3
[200]	training's rmse: 12562.1	valid_1's rmse: 18393.1
[210]	training's rmse: 

/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['D', 'K', 'L', 'S', '定期借家', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument 

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001039 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2087
[LightGBM] [Info] Number of data points in the train set: 25176, number of used features: 14
[LightGBM] [Info] Start training from score 118197.706188
Training until validation scores don't improve for 10 rounds
[10]	training's rmse: 49200.5	valid_1's rmse: 51074.2
[20]	training's rmse: 35674.7	valid_1's rmse: 37131.5
[30]	training's rmse: 27892.9	valid_1's rmse: 28948.1
[40]	training's rmse: 23285.5	valid_1's rmse: 24097.4
[50]	training's rmse: 20546	valid_1's rmse: 21455
[60]	training's rmse: 18685.9	valid_1's rmse: 19893.7
[70]	training's rmse: 17641.1	valid_1's rmse: 19082.2
[80]	training's rmse: 16838.4	valid_1's rmse: 18545.3
[90]	training's rmse: 16219.4	valid_1's rmse: 18088.9
[100]	training's rmse: 15743.6	valid_1's rm

,importance
面積,4.764802e+14
L,3.991274e+14
平均地価,2.100572e+14
最寄り駅,1.156990e+14
築年数,7.025105e+13
全体の階数,3.528585e+13
契約期間,1.288053e+13
所在階,1.000691e+13
所要時間,7.106218e+12
定期借家,3.527375e+12


チューニングを行う

In [260]:
X_train = pd.concat([house_age, area_size, contract_span2, reg_rent, floor_scores2, Floor_scores2, stations, minits, land_aveprice, room_arrange[["部屋数","L","D","K",'S']]], axis=1)
y_train = rent
#X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=0)
X_test = pd.concat([test_house_age, test_area_size, test_contract_span2, test_reg_rent, test_floor_scores2, test_Floor_scores2, test_stations, test_minits, test_land_aveprice, test_room_arrange[["部屋数","L","D","K","S"]]], axis=1)
category_lists = ['定期借家', '最寄り駅', 'L', 'D', 'K', 'S']

lcv = KFold(n_splits=5, shuffle=True, random_state=0)
rmse_scores = []
X_train['KFold'] = -1
#foldはvalidの対象となる値で０から４
for fold_id, (train_index, valid_index) in enumerate(cv.split(X_train)):
    X_train.loc[valid_index,'KFold'] = fold_id
    y_train.loc[valid_index,'KFold'] = fold_id

def objective(trial):
    #foldはvalidの対象となる値で０から４
    for fold in range(5):
        X_tr = X_train[X_train['KFold']!=fold].drop('KFold',axis=1)
        X_val = X_train[X_train['KFold']==fold].drop('KFold',axis=1)
        y_tr = y_train[y_train['KFold']!=fold].drop('KFold',axis=1)
        y_val = y_train[y_train['KFold']==fold].drop('KFold',axis=1)

        lgb_train = lgb.Dataset(X_tr, y_tr)
        lgb_eval = lgb.Dataset(X_val, y_val, reference=lgb_train)
        params = {
            'objective':'regression',
            'metrics':'rmse',
            'learning_rate':0.05,
            'reg_lambda': trial.suggest_float('reg_lambda', 0.0000001, 0.0001, log=True),
            'max_bin': trial.suggest_int('max_bin', 255, 600),
            'num_leaves': trial.suggest_int('num_leaves', 50, 128),
        }

        model = lgb.train(
                            params,
                            lgb_train, 
                            valid_sets=[lgb_train, lgb_eval], 
                            verbose_eval=10, 
                            num_boost_round=3000, 
                            early_stopping_rounds=10,
                            categorical_feature = category_lists
                            )

        y_pred_val = model.predict(X_val, num_iteration=model.best_iteration)
        rmse_score = mean_squared_error(y_val, y_pred_val, squared=False)
    loss = np.mean(rmse_score)
    return loss

In [261]:
study = optuna.create_study(sampler=optuna.samplers.RandomSampler(seed=0))
study.optimize(objective, n_trials=40)

[I 2022-09-13 17:25:46,280] A new study created in memory with name: no-name-47d8acf3-b2ac-40f5-bea7-c764d739e5c3
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['D', 'K', 'L', 'S', '定期借家', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of Lig

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001267 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2087
[LightGBM] [Info] Number of data points in the train set: 25176, number of used features: 14
[LightGBM] [Info] Start training from score 118516.069987
Training until validation scores don't improve for 10 rounds
[10]	training's rmse: 48825.6	valid_1's rmse: 52082.5
[20]	training's rmse: 34524.9	valid_1's rmse: 40394.8
[30]	training's rmse: 26214.1	valid_1's rmse: 34203.7
[40]	training's rmse: 21069.2	valid_1's rmse: 30765.7
[50]	training's rmse: 17930.2	valid_1's rmse: 28811.3
[60]	training's rmse: 16072.1	valid_1's rmse: 27695.2
[70]	training's rmse: 14724.7	valid_1's rmse: 26889.9
[80]	training's rmse: 13821.2	valid_1's rmse: 26402.4
[90]	training's rmse: 13123.4	valid_1's rmse: 26018.3
[100]	training's rmse: 12502.2	valid_1'

/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['D', 'K', 'L', 'S', '定期借家', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument 

[10]	training's rmse: 49473	valid_1's rmse: 49579.1
[20]	training's rmse: 35601.4	valid_1's rmse: 36073.9
[30]	training's rmse: 27772.3	valid_1's rmse: 28638.7
[40]	training's rmse: 23301.4	valid_1's rmse: 24340.9
[50]	training's rmse: 20542.8	valid_1's rmse: 21621
[60]	training's rmse: 18965.2	valid_1's rmse: 20218.9
[70]	training's rmse: 17858.6	valid_1's rmse: 19554.6
[80]	training's rmse: 17114.6	valid_1's rmse: 19173.3
[90]	training's rmse: 16419.5	valid_1's rmse: 18883.5
[100]	training's rmse: 15878.8	valid_1's rmse: 18708
[110]	training's rmse: 15397.3	valid_1's rmse: 18619.7
[120]	training's rmse: 14981.9	valid_1's rmse: 18606.1
Early stopping, best iteration is:
[111]	training's rmse: 15350.3	valid_1's rmse: 18587.1
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001162 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2087
[L

/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['D', 'K', 'L', 'S', '定期借家', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument 

...

[10]	training's rmse: 49418	valid_1's rmse: 48010.7
[20]	training's rmse: 35233.3	valid_1's rmse: 35566.8
[30]	training's rmse: 26999.1	valid_1's rmse: 28414.2
[40]	training's rmse: 22315.9	valid_1's rmse: 24802.6
[50]	training's rmse: 19603.7	valid_1's rmse: 22875
[60]	training's rmse: 18018.5	valid_1's rmse: 21948.2
[70]	training's rmse: 16998.4	valid_1's rmse: 21312.2
[80]	training's rmse: 16285	valid_1's rmse: 20810.9
[90]	training's rmse: 15713.7	valid_1's rmse: 20385
[100]	training's rmse: 15159.7	valid_1's rmse: 20076.9
[110]	training's rmse: 14674.6	valid_1's rmse: 19905.9
[120]	training's rmse: 14234.4	valid_1's rmse: 19792.6
[130]	training's rmse: 13822.6	valid_1's rmse: 19580.8
[140]	training's rmse: 13418.5	valid_1's rmse: 19420.5
[150]	training's rmse: 13083	valid_1's rmse: 19309.9
[160]	training's rmse: 12749.3	valid_1's rmse: 19225.2
[170]	training's rmse: 12487.4	valid_1's rmse: 19149.5
[180]	training's rmse: 12197.3	valid_1's rmse: 19092.2
[190]	training's rmse: 11996	

/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['D', 'K', 'L', 'S', '定期借家', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument 

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001086 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2246
[LightGBM] [Info] Number of data points in the train set: 25176, number of used features: 14
[LightGBM] [Info] Start training from score 118197.706188
Training until validation scores don't improve for 10 rounds
[10]	training's rmse: 48941.3	valid_1's rmse: 50890.5
[20]	training's rmse: 35356.5	valid_1's rmse: 37000.5
[30]	training's rmse: 27490	valid_1's rmse: 28525.1
[40]	training's rmse: 22995.8	valid_1's rmse: 23770.3
[50]	training's rmse: 20145.6	valid_1's rmse: 21062.1
[60]	training's rmse: 18492.2	valid_1's rmse: 19678.4
[70]	training's rmse: 17421.7	valid_1's rmse: 18897
[80]	training's rmse: 16635.3	valid_1's rmse: 18466.4
[90]	training's rmse: 16036.9	valid_1's rmse: 18183.9
[100]	training's rmse: 15578.5	valid_1's rm

[I 2022-09-13 17:32:09,027] Trial 39 finished with value: 17100.235333699613 and parameters: {'reg_lambda': 5.543941231681784e-06, 'max_bin': 560, 'num_leaves': 104}. Best is trial 32 with value: 15648.736376362416.


[230]	training's rmse: 12278.4	valid_1's rmse: 17107.3
Early stopping, best iteration is:
[221]	training's rmse: 12406.1	valid_1's rmse: 17100.2


In [262]:
study.best_params

{'reg_lambda': 5.748291778269802e-06, 'max_bin': 261, 'num_leaves': 115}

チューニング結果を試してみる

In [272]:
df = pd.read_csv('../data/outer_data/train.csv')
dummy_address_aveprice = address_aveprice['平均地価']
land_aveprice = dummy_address_aveprice.iloc[:len(df),:]
test_land_aveprice = dummy_address_aveprice.iloc[len(df):,:]
land_aveprice.columns = ['平均地価']
test_land_aveprice.columns = ['平均地価']
X_train = pd.concat([house_age, area_size, contract_span2, reg_rent, floor_scores2, Floor_scores2, stations, minits, land_aveprice, room_arrange[["部屋数","L","D","K","S"]]], axis=1)
y_train = rent
X_test = pd.concat([test_house_age, test_area_size, test_contract_span2, test_reg_rent, test_floor_scores2, test_Floor_scores2, test_stations, test_minits, test_land_aveprice, test_room_arrange[["部屋数","L","D","K","S"]]], axis=1)
category_lists = ['定期借家','最寄り駅', 'L', 'D','K','S']

In [283]:
cv = KFold(n_splits=5, shuffle=True, random_state=0)
for fold_id, (train_index, valid_index) in enumerate(cv.split(X_train)):
    X_tr = X_train.loc[train_index,:]
    X_val = X_train.loc[valid_index, :]
    y_tr = y_train.loc[train_index]
    y_val = y_train.loc[valid_index]

    lgb_train = lgb.Dataset(X_tr, y_tr)
    lgb_eval = lgb.Dataset(X_val, y_val, reference=lgb_train)

    params = {
        'objective':'regression',
        'metrics':'rmse',
        'learning_rate':0.05,
        'reg_lambda':  5.748291778269802e-06,
        'max_bin': 261,
        'num_leaves': 115
    }

    model = lgb.train(
                        params,
                        lgb_train, 
                        valid_sets=[lgb_train, lgb_eval], 
                        verbose_eval=10, 
                        num_boost_round=1000, 
                        early_stopping_rounds=10,
                        categorical_feature = category_lists
                        )

y_pred = model.predict(X_test, num_iteration=model.best_iteration)
# feature importanceを表示
importance = pd.DataFrame(model.feature_importance(importance_type = "gain"), index=X_train.columns, columns=['importance'])
df_importance = pd.DataFrame(importance)
df_importance = df_importance.sort_values('importance', ascending=False) # 降順ソート
display(df_importance)


/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['D', 'K', 'L', 'S', '定期借家', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument 

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001247 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1346
[LightGBM] [Info] Number of data points in the train set: 25176, number of used features: 14
[LightGBM] [Info] Start training from score 118516.069987
Training until validation scores don't improve for 10 rounds
[10]	training's rmse: 48718.2	valid_1's rmse: 52172.2
[20]	training's rmse: 34556.3	valid_1's rmse: 40665.7
[30]	training's rmse: 26158.4	valid_1's rmse: 34433.7
[40]	training's rmse: 21208.7	valid_1's rmse: 31009.4
[50]	training's rmse: 17974.4	valid_1's rmse: 28902
[60]	training's rmse: 16139	valid_1's rmse: 27724
[70]	training's rmse: 14805.5	valid_1's rmse: 26956.9
[80]	training's rmse: 13816.7	valid_1's rmse: 26378.3
[90]	training's rmse: 13002	valid_1's rmse: 25898.9
[100]	training's rmse: 12406.4	valid_1's rmse: 

/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['D', 'K', 'L', 'S', '定期借家', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument 

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000850 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1346
[LightGBM] [Info] Number of data points in the train set: 25176, number of used features: 14
[LightGBM] [Info] Start training from score 118478.809819
Training until validation scores don't improve for 10 rounds
[10]	training's rmse: 49286.1	valid_1's rmse: 49298.3
[20]	training's rmse: 35426.5	valid_1's rmse: 35802.4
[30]	training's rmse: 27501.9	valid_1's rmse: 28271.9
[40]	training's rmse: 23022.1	valid_1's rmse: 24197.7
[50]	training's rmse: 20203.3	valid_1's rmse: 21486.2
[60]	training's rmse: 18571.2	valid_1's rmse: 19960.1
[70]	training's rmse: 17333.2	valid_1's rmse: 19016.1
[80]	training's rmse: 16459.8	valid_1's rmse: 18480.8
[90]	training's rmse: 15722	valid_1's rmse: 18107.6
[100]	training's rmse: 15112.4	valid_1's 

/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['D', 'K', 'L', 'S', '定期借家', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument 

...

/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['D', 'K', 'L', 'S', '定期借家', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument 

[10]	training's rmse: 49576.4	valid_1's rmse: 47857.7
[20]	training's rmse: 35563.1	valid_1's rmse: 35133.2
[30]	training's rmse: 27516.3	valid_1's rmse: 28332.8
[40]	training's rmse: 22797.4	valid_1's rmse: 24621.9
[50]	training's rmse: 19910.6	valid_1's rmse: 22670.3
[60]	training's rmse: 18217.1	valid_1's rmse: 21700.7
[70]	training's rmse: 16970.5	valid_1's rmse: 20921.4
[80]	training's rmse: 16097.4	valid_1's rmse: 20482
[90]	training's rmse: 15408.9	valid_1's rmse: 20184.9
[100]	training's rmse: 14831.2	valid_1's rmse: 19811.3
[110]	training's rmse: 14335.8	valid_1's rmse: 19516.7
[120]	training's rmse: 13911.6	valid_1's rmse: 19231.8
[130]	training's rmse: 13525.1	valid_1's rmse: 19032.7
[140]	training's rmse: 13167.7	valid_1's rmse: 18901.2
[150]	training's rmse: 12856	valid_1's rmse: 18783.6
[160]	training's rmse: 12562.5	valid_1's rmse: 18738.4
[170]	training's rmse: 12304.4	valid_1's rmse: 18634.1
[180]	training's rmse: 12068.1	valid_1's rmse: 18591
Early stopping, best iter

/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['D', 'K', 'L', 'S', '定期借家', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument 

[20]	training's rmse: 35512.2	valid_1's rmse: 36918.9
[30]	training's rmse: 27648.7	valid_1's rmse: 28439.8
[40]	training's rmse: 23262.9	valid_1's rmse: 23748.2
[50]	training's rmse: 20374.8	valid_1's rmse: 21019.6
[60]	training's rmse: 18551.6	valid_1's rmse: 19458.4
[70]	training's rmse: 17305.4	valid_1's rmse: 18648.8
[80]	training's rmse: 16430	valid_1's rmse: 18160.2
[90]	training's rmse: 15745.2	valid_1's rmse: 17903.1
[100]	training's rmse: 15185.5	valid_1's rmse: 17648.1
[110]	training's rmse: 14718.5	valid_1's rmse: 17435.9
[120]	training's rmse: 14295	valid_1's rmse: 17214.3
[130]	training's rmse: 13870.8	valid_1's rmse: 16978.3
[140]	training's rmse: 13535.6	valid_1's rmse: 16853.2
[150]	training's rmse: 13212.7	valid_1's rmse: 16725.6
[160]	training's rmse: 12918.2	valid_1's rmse: 16636.7
[170]	training's rmse: 12614	valid_1's rmse: 16473.2
[180]	training's rmse: 12368.5	valid_1's rmse: 16396.9
[190]	training's rmse: 12100	valid_1's rmse: 16304.6
[200]	training's rmse: 118

,importance
面積,4.550062e+14
L,4.078170e+14
平均地価,2.166120e+14
最寄り駅,1.188204e+14
築年数,7.592671e+13
全体の階数,3.625630e+13
契約期間,1.293806e+13
所在階,1.274474e+13
所要時間,9.060635e+12
定期借家,5.275432e+12


In [291]:
df = pd.read_csv('../data/outer_data/test.csv')
id = df['id']
y_pred = pd.DataFrame(y_pred)
result = pd.concat([id, y_pred],axis=1)
result.to_csv('../data/result_data/LGBM_cv_result.csv',index=False)